# Práctica 3: Representación del conocimiento

__Fecha de entrega: 23 de febrero de 2023__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Nombres de los estudiantes: Fernando Isaías Leal Sánchez y Jinqing Cai__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural). 

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.
 
Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [ ]:
## Lo primero sería instalar en tu entorno de Python wdsparql
## Si no lo tienes instalado, puedes hacerlo desde el notebook
import sys

!{sys.executable} -m pip install wdsparql

# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.4 MB/s eta 0:00:00


### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [ ]:
%%wdsparql
SELECT ?item ?itemLabel 
WHERE 
{
    ?item wdt:P31 wd:Q2934.  # item es una instancia de (P31) la clase Cabra (Q2934)
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,item,itemLabel
0,http://www.wikidata.org/entity/Q29545,Kevin Durant
1,http://www.wikidata.org/entity/Q151345,Billygoat Hennes
2,http://www.wikidata.org/entity/Q551597,Jules Vallès
3,http://www.wikidata.org/entity/Q3569037,William Windsor
4,http://www.wikidata.org/entity/Q13050139,Q13050139
5,http://www.wikidata.org/entity/Q23003932,His Whiskers
6,http://www.wikidata.org/entity/Q24287064,Taffy
7,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
8,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
9,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I


### Consulta 1

[U2 (Q396)](https://www.wikidata.org/wiki/Q396) es una banda de rock con una larga trayectoria y numerosos éxitos. 

Vamos a comenzar por averiguar su fecha y lugar de formación (localidad y país).

In [ ]:
%%wdsparql
SELECT ?fecha ?paisLabel ?localidadLabel
WHERE {
    wd:Q396 wdt:P17 ?pais; # U2(Q396) tiene país(P17) ?pais
            wdt:P571 ?fecha; #        y fecha de creacion (P571)
            wdt:P740 ?localidad. #    y lugar de formación (P740)
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,fecha,paisLabel,localidadLabel
0,1976-01-01T00:00:00Z,Republic of Ireland,Dublin


### Consulta 2

A continuación vamos a averiguar todos los miembros que se le conocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por su apellido e incluir su nombre completo y el artístico.

Puede que la siguiente información sobre el uso de [OPTIONAL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#OPTIONAL) te resulte útil.

In [ ]:
%%wdsparql
SELECT ?nombreCompleto ?nombreArtistico ?familyNameLabel
WHERE {
    ?member wdt:P463 wd:Q396;           # ?member es miembro (P463) del U2 (Q396)       
            wdt:P734 ?familyName.       #         y tiene familyname (P734) ?familyName
    
    OPTIONAL { 
        # Opcionalmente, tiene un nombre artistico (pseudonym, P742). Sin optional no incluirá aquellos miembros sin nombre artistico
        ?member wdt:P742 ?nombreArtistico. 
    }
    
    OPTIONAL { 
        ?member wdt:P1477 ?nombreCompleto.  # y tiene birthname (P1477) ?nombreCompleto
    } 

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?familyNameLabel) # Ordenar de alfabeticamente por el nombre de familia (apellido)

,nombreCompleto,nombreArtistico,familyNameLabel
0,Adam Charles Clayton,None,Clayton
1,David Howell Evans,The Edge,Evans
2,Paul David Hewson,Bono,Hewson
3,Larry Mullen Jr,None,Mullen


Es importante usar dos OPTIONAL distintos para que se muestren todos los miembros, tengan o no pseudónimo y nombre completo asociados.

### Consulta 3

[Bono (Q834621)](http://www.wikidata.org/entity/Q834621) es el cantante de la banda. Además de cantante, ha tenido muchas otras ocupaciones. ¿Cuáles de ellas están relacionadas directamente con el mundo del [arte (Q483501)](https://www.wikidata.org/wiki/Q483501)? Ten en cuenta que la jerarquía de ocupaciones puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

In [ ]:
%%wdsparql
SELECT ?ocupacionLabel
WHERE {
    # Hemos usado bind para que se entienda mejor, en vez de usar wdt:PXXX que es difícil de ver a simple vista
    bind(wd:Q834621 as ?bono).
    bind(wdt:P106 as ?workedAs).
    bind(wd:Q483501 as ?artist).
  
    ?bono ?workedAs ?ocupacion.        # Bono ha trabajado en ?ocupacion
    ?ocupacion wdt:P279+ ?artist.      # Solo las ocupaciones que son subclass(P279) del artist o subclass indirecto de artist
  
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,ocupacionLabel
0,screenwriter
1,actor
2,singer-songwriter
3,guitarist


Si no añadimos el `+` a la relación `wdt:P279` muchas de las profesiones no se recuperan.

### Consulta 4

Bono se ha casado una vez, con Ali Hewson, con la que tiene varios hijos. Queremos conocer los nombres de esos hijos y sus fechas de nacimiento. Los resultados deben aparecer ordenados cronológicamente.

In [ ]:
%%wdsparql
SELECT ?childLabel ?fechaNacimiento      # childLabel ya da el nombre completo del hijo
WHERE {
    # ID de nodos de Bono y Ali Hewson
    bind(wd:Q834621 as ?bono).
    bind(wd:Q777287 as ?ali).

    # Propiedades de ser padre, madre y tener fecha de cumpleaños
    bind(wdt:P22 as ?father).
    bind(wdt:P25 as ?mother).
    bind(wdt:P569 as ?birthdate).
    
    ?child ?father ?bono; # child tiene como padre Bono
           ?mother ?ali; # Y como madre Ali
           ?birthdate ?fechaNacimiento; # Y la fecha del nacimiento asociado por arista P569
    
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?fechaNacimiento) # Ordenar de menor a mayor fecha de nacimiento

,childLabel,fechaNacimiento
0,Eve Hewson,1991-07-07T00:00:00Z
1,Elijah Hewson,1999-08-18T00:00:00Z


### Consulta 5

Bono ha recibido muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha en la que fue premiado (si está disponible). Los resultados se deben mostrar ordenado por fecha.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [ ]:
%%wdsparql
SELECT ?prizeNameLabel ?fecha
WHERE {
    bind(wd:Q834621 as ?bono).
    bind(p:P166 as ?won).      # award received

    ?bono ?won ?prize.         # prize es el statement node del hecho de ganar un premio
    ?prize ps:P166 ?prizeName. # Aqui hemos tenido que usar ps en vez de wdt, porque prize es un statement node, y tenemos que extraer el nombre (objeto del statement) usando ps
    OPTIONAL { 
        ?prize pq:P585 ?fecha. # aplicar el cualificador de point in time (P585)
    } # Opcionalmente tiene una fecha, se extraye con pq (property qualifier)

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?fecha) # Ordenado por fecha de menor a mayor

,prizeNameLabel,fecha
0,Grammy Awards,None
1,Fellow of the American Academy of Arts and Sci...,None
2,MusiCares Person of the Year,2003-01-01T00:00:00Z
3,Knight of the Legion of Honour,2003-01-01T00:00:00Z
4,TED Prize,2005-01-01T00:00:00Z
5,Rock and Roll Hall of Fame,2005-01-01T00:00:00Z
6,Officer of the Order of Liberty,2005-04-21T00:00:00Z
7,Time Person of the Year,2005-12-19T00:00:00Z
8,Philadelphia Liberty Medal,2007-01-01T00:00:00Z
9,NAACP Image Award – Chairman's Award,2007-01-01T00:00:00Z


### Consulta 6

Ahora queremos conocer todos los discos de U2 del tipo [albunes en directo (Q209939)](https://www.wikidata.org/wiki/Q209939) y cualquiera de sus subgéneros. Los resultados se deben mostrar ordenados por nombre e incluir el año de publicación.

In [ ]:
%%wdsparql
SELECT DISTINCT ?albumLabel ?genreLabel (YEAR(?publishDate) as ?year)
WHERE {
  BIND(wd:Q209939 as ?liveAlbum).

  ?album wdt:P175 wd:Q396;                # ?album tiene performer (P175) a U2
         wdt:P7937/wdt:P279* ?liveAlbum;  #        su form of creative work (P7937) es subclass of (P279) live album (o indirectamente, por medio de otras subclases)
         wdt:P577 ?publishDate;           #        su fecha de publicacion (P577)
         wdt:P136 ?genre.                 #        su genéro (P136)  

  BIND(SAMPLE(?genre) AS ?genreSample). # Tomamos cualquiera de los géneros, y lo llamamos genreSample

  SERVICE wikibase:label { 
    bd:serviceParam wikibase:language "en".
    ?album        rdfs:label ?albumLabel. # Hemos incluido estos por que habia fallos para obtener label de genreSample
    ?genreSample  rdfs:label ?genreLabel.
  }
}
GROUP BY ?album ?albumLabel ?publishDate ?genreLabel  # publishDate tambien porque sino piensa que puede haber multiples y da error al no usar funcion de agregacion
ORDER BY ASC(?albumLabel)

,albumLabel,genreLabel,year
0,360° at the Rose Bowl,rock music,2010
1,Live from the Point Depot,rock music,2004
2,Rattle and Hum,rock music,1988
3,The Complete U2,rock music,2004
4,"U2 Go Home: Live from Slane Castle, Ireland",rock music,2003
5,U2.COMmunication,rock music,2005
6,U218 Singles,rock music,2006
7,U22,rock music,2012
8,U2: Rattle and Hum,rock music,1988
9,Under a Blood Red Sky,rock music,1983


### Consulta 7

Queremos saber si hay obras de U2 publicadas varias veces (en diferentes años). Para ello vamos a recuperar las obras de tipo [lanzamiento de un grupo (group release)](https://www.wikidata.org/wiki/Q108346082) pero mostrando también la fecha de publicación más antigua de cada una y el número de apariciones. Los resultados deben aparecer en orden cronológico.

Para calcular la fecha correcta tendrás que agrupar las respuestas por album y aplicar una función de agregación sobre las fechas de lanzamiento. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Grouping) te resulte útil.

In [ ]:
%%wdsparql
SELECT ?albumLabel (MIN(?publishDate) as ?date) (COUNT(*) as ?count) {
    ?album wdt:P175 wd:Q396;                    # ?album tiene performer (P175) a U2 (Q396)
           (wdt:P31/wdt:P279*) wd:Q108346082;   #        y es una instancia de (P31) release group (Q108346082), o instancia de una subclase (puede ser varias relaciones de subclase, P279) de release group
           wdt:P577 ?publishDate;               #        y tiene fecha de publicacion (P577)
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?album ?albumLabel # En un principio no hay varios albumLabel para el mismo album, pero si no ponemos eso da error de agregacion (no sabe que es unico)
ORDER BY ASC(?date) # Ordenar de menor a mayor fecha

,albumLabel,date,count
0,Three,1979-09-26T00:00:00Z,1
1,Another Day,1980-02-26T00:00:00Z,1
2,11 O'Clock Tick Tock,1980-05-16T00:00:00Z,1
3,A Day Without Me,1980-08-18T00:00:00Z,1
4,Boy,1980-10-20T00:00:00Z,1
5,I Will Follow,1980-10-24T00:00:00Z,1
6,Fire,1981-07-01T00:00:00Z,1
7,Gloria,1981-10-05T00:00:00Z,1
8,October,1981-10-12T00:00:00Z,1
9,A Celebration,1982-03-22T00:00:00Z,1


### Consulta 8

U2 es sin duda un grupo con una larga trayectoria. ¿Cuántas discos ha lanzado? Vamos a considerar sólo instancias directas de [album de estudio (Q208569)](https://www.wikidata.org/wiki/Q208569).

In [ ]:
%%wdsparql
SELECT (COUNT(*) as ?num_studio_album)
WHERE {
    ?album wdt:P175 wd:Q396;                  # ?ablum tiene como performancer (P175) a U2 (Q396)
           wdt:P7937/wdt:P279* wd:Q208569;    #        su form of creative work (P7937) es una instancia (P279) de studio album (Q208569) (o indirectamente, por medio de otras subclases) 
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,num_studio_album
0,14


### Consulta 9

Entre todos sus discos de estudio, queremos recuperar los títulos, fechas de lanzamiento y discográfica de los discos de la década de los 1990. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

In [ ]:
%%wdsparql
SELECT ?albumLabel ?publishDate ?discograficaLabel {
    ?album wdt:P175 wd:Q396;                        # ?album tiene performancer (P175) a U2 (Q396)
           wdt:P7937/wdt:P279* wd:Q208569;          #         de tipo studio album, como el ejercicio anterior
           wdt:P577 ?publishDate;                   #         con fecha de publicacion (P577) ?publishDate
           wdt:P264 ?discografica;                  #         tiene discografica (record label P264)

    BIND(YEAR(?publishDate) AS ?year).              # Quedar con el ano de la fecha de publicacion
    FILTER(1990 <= ?year && ?year <= 1999).         # Y solo quedamos con aquellos discos con fecha de publicacion de decada de 90
    
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,albumLabel,publishDate,discograficaLabel
0,Zooropa,1993-07-05T00:00:00Z,Island Records
1,Achtung Baby,1991-11-18T00:00:00Z,Island Records
2,Pop,1997-03-03T00:00:00Z,Island Records


### Consulta 10

¿Qué otros personajes famosos nacieron el mismo año que Bono y tiene su misma nacionalidad? Para cada uno de ellos muestra su nombre y ocupación. Muestra los resultados ordenados por nombre y fecha de nacimiento.

In [ ]:
%%wdsparql
SELECT ?personLabel (GROUP_CONCAT(?ocupationLabel; separator=", ") as ?ocupation)
WHERE {
    bind(wd:Q834621 as ?bono).
    bind(wdt:P569 as ?dateOfBirth).
  
    ?bono wdt:P569 ?birthdateBono;                # Primero consultamos la fecha de nacimiento (P569)
          wdt:P27 ?nationalityBono.               #   y la nacionalidad (country of citizenship P27) de Bono
  
    ?person wdt:P569 ?birthdate;                  # Utilizando la informacion anterior, buscamos personas con misma nacionalidad que Bono
            wdt:P27 ?nationalityBono;
            wdt:P106 ?ocupation
    
    FILTER(YEAR(?birthdateBono) = YEAR(?birthdate)) # Filtramos a parte el año de nacimiento, porque lo que se devuelve en la consulta es la fecha completa, y no año
    
    SERVICE wikibase:label { 
        bd:serviceParam wikibase:language "en".
        ?person      rdfs:label ?personLabel .
        ?ocupation   rdfs:label ?ocupationLabel .
    }
}
GROUP BY ?person ?personLabel # Agrupar por persona, y label por el mismo motivo de antes

,personLabel,ocupation
0,Roma Downey,"actor, film producer, film actor"
1,Brian Cowen,"lawyer, politician, diplomat, Gaelic football ..."
2,Packie Bonner,association football player
3,Trevor Sargent,"politician, Esperantist, environmentalist"
4,Paul Harrington,"singer, musician"
5,David Keating,"screenwriter, film director, film producer"
6,Bono,"screenwriter, actor, entrepreneur, singer-song..."
7,Michael Martin,"politician, diplomat"
8,Christy Dignam,musician
9,Martin Sludds,golfer


### Consulta 11

¿Cuántos asertos hay sobre U2 en Wikidata? Ten en cuenta que puede aparece tanto como sujeto como objeto de cada tripleta.

In [ ]:
%%wdsparql
SELECT DISTINCT (count(*) as ?count)
WHERE {
    bind(wd:Q396 as ?u2).
  
    {?u2 ?p1 ?o}     # Cuando U2 (Q269) es subjeto de una propiedad ?p1, con objeto ?o
    UNION
    {?s ?p2 ?u2.}    # Cuando U2 (Q269) es objeto de una propiedad ?p2 con sujeto ?s

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,count
0,1432


__Fecha de las consultas: 21 de abril de 2022__